# Content based recommendation-Plot description


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import pickle
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from stop_words import get_stop_words
import gensim
from gensim import corpora, models,similarities
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import random
import tqdm
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import pprint

In [2]:
df = pd.read_csv('corpus.csv')
print(df.shape)
df.head(10)

(4432, 4)


,Article_id,Title,Content,url
0,0,"TVs, Tablets And Phones: How To Manage 'Screen...",In today’s media-rich world (or media-saturate...,https://www.ibtimes.com/tvs-tablets-phones-how...
1,1,Firefighters Warn Hand Sanitizers Left Inside ...,KEY POINTS Fire fighters warn that hand saniti...,https://www.ibtimes.com/firefighters-warn-hand...
2,2,Hand Sanitizers: Do They Go Out Of Date?,KEY POINTS Coronavirus confirmed cases around ...,https://www.ibtimes.com/hand-sanitizers-do-the...
3,3,Coronavirus Update: SpaceX Is Creating Hand Sa...,KEY POINTS SpaceX is making face shields and h...,https://www.ibtimes.com/coronavirus-update-spa...
4,4,Trump Says US Won't Close If Hit By Coronaviru...,KEY POINTS Trump will not shut down America ag...,https://www.ibtimes.com/trump-says-us-wont-clo...
5,5,Coronavirus Vaccine: US To Pay $1.2B For 300 M...,"KEY POINTS The candidate COVID-19 vaccine, AZD...",https://www.ibtimes.com/coronavirus-vaccine-us...
6,6,"US Coronavirus Deaths Hit 96,228; COVID-19 Now...",KEY POINTS Tom Ridge says the world will never...,https://www.ibtimes.com/us-coronavirus-deaths-...
7,7,3-Month-Old Louisiana Baby Hospitalized With F...,Louisiana police are searching for a 24-year-o...,https://www.ibtimes.com/3-month-old-louisiana-...
8,8,Missing Connecticut Mom Jennifer Dulos Update:...,May 24 will mark the one-year anniversary of J...,https://www.ibtimes.com/missing-connecticut-mo...
9,9,Ohio Child Abuse: 6-Year-Old Slammed To Ground...,An Ohio man was arrested and charged with chil...,https://www.ibtimes.com/ohio-child-abuse-6-yea...


In [3]:
df = df[['Article_id', 'Title', 'Content']].dropna()
df = df.drop_duplicates(keep='first')
df.shape

(4427, 3)

In [4]:
texts=[] # Define an array for holding the relevant words in each documents

# Now, change the content column of the dataframe to a list
content = df['Content'].tolist()

# Define the common stop words in english using the stop_words library
en_stop = get_stop_words('en')
# Define other words or letters that may be repetitive/irrelevant
letters = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]
other = ["wa","ha","one","two","id","re","http","com","mr","image","photo","caption","don","sen","pic","co",
         "source","watch","play","duration","video","momentjs","getty","images","newsletter", "story", "go", "like", "say",
         "will", "just", "today", "can", "year", "make", "view", "read","said", "also"]

# Define the stemmer
stemmer = SnowballStemmer("english")

# Select each doocument separately and perform preprocessing
for i in range(len(content)):
    doc = content[i]

    doc = doc.replace("\n"," ")
    doc = doc.replace("\'","")

    doc = gensim.utils.simple_preprocess(doc)
    
    # Lemmatize the document
    doc = [stemmer.stem(WordNetLemmatizer().lemmatize(word, pos='v')) for word in doc]
    
    # Remove stop words
    doc = [word for word in doc if not word in (en_stop+letters+other)]
    
    texts.append(doc)

In [5]:
#We now split the pre-processed corpus into training and testing data sets
random.seed(75) # For reproducibility
train_indices = random.sample(list(range(0,len(texts))),len(texts)-1000)
test_indices = [x for x in list(range(0,len(texts))) if x not in train_indices]

train_data = [texts[i] for i in train_indices]
test_data = [texts[i] for i in test_indices]

In [6]:
dictionary = gensim.corpora.Dictionary(train_data)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in train_data]

In [7]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=17, id2word = dictionary, passes=50)

In [8]:
ldamodel.print_topics()

[(0,
  '0.018*"diamond" + 0.013*"ring" + 0.009*"studi" + 0.009*"drug" + 0.008*"cut" + 0.008*"patient" + 0.007*"research" + 0.007*"use" + 0.006*"treatment" + 0.006*"engag"'),
 (1,
  '0.013*"coronavirus" + 0.013*"trump" + 0.010*"countri" + 0.010*"covid" + 0.009*"china" + 0.009*"virus" + 0.009*"world" + 0.008*"pandem" + 0.008*"state" + 0.008*"peopl"'),
 (2,
  '0.015*"polic" + 0.008*"minist" + 0.007*"govern" + 0.006*"peopl" + 0.006*"parti" + 0.005*"home" + 0.005*"kill" + 0.005*"offici" + 0.005*"leader" + 0.005*"alleg"'),
 (3,
  '0.009*"game" + 0.006*"get" + 0.006*"new" + 0.005*"take" + 0.005*"show" + 0.005*"time" + 0.005*"star" + 0.004*"famili" + 0.004*"reveal" + 0.004*"old"'),
 (4,
  '0.008*"work" + 0.008*"use" + 0.005*"need" + 0.005*"time" + 0.005*"peopl" + 0.004*"new" + 0.004*"servic" + 0.004*"busi" + 0.004*"technolog" + 0.004*"compani"'),
 (5,
  '0.010*"flight" + 0.008*"airlin" + 0.007*"airport" + 0.007*"worth" + 0.006*"lee" + 0.006*"start" + 0.006*"polic" + 0.005*"aircraft" + 0.005*"a

In [9]:
# Visualize the topics
vis = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)
vis

C:\Users\Sasidharan\Anaconda2\envs\py36\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.086414 -0.052250       1        1  12.615986
10     0.146276 -0.082581       2        1   9.948128
4      0.080686  0.083497       3        1   9.437263
13     0.055040  0.009906       4        1   7.955089
2      0.051177 -0.139284       5        1   7.937665
3     -0.110654  0.087855       6        1   7.069884
6     -0.092532  0.106438       7        1   6.967463
12     0.137202  0.041704       8        1   6.051895
15    -0.074046  0.034739       9        1   6.047397
0      0.061366  0.194227      10        1   4.983050
7      0.045326  0.057937      11        1   4.590960
14     0.055878  0.006406      12        1   3.709087
16    -0.023599 -0.077939      13        1   3.048744
11    -0.145933  0.061770      14        1   2.980412
9      0.014598 -0.156725      15        1   2.921864
8     -0.174875 -0.122871      16        1   1.993497
5     -0.112327 -0.052829      17        1   1.741617, topic_info=     Category         Freq         Term        Total  loglift  logprob
83    Default  1683.000000        trump  1683.000000  30.0000  30.0000
1710  Default  1193.000000           rs  1193.000000  29.0000  29.0000
13    Default  1976.000000  coronavirus  1976.000000  28.0000  28.0000
9976  Default   758.000000      diamond   758.000000  27.0000  27.0000
63    Default  1611.000000       presid  1611.000000  26.0000  26.0000
...       ...          ...          ...          ...      ...      ...
409   Topic17    51.294464        sever   890.080688   1.1966  -5.6322
190   Topic17    49.647831        offic   914.101318   1.1374  -5.6649
140   Topic17    47.070107       famili   767.262817   1.2592  -5.7182
229   Topic17    45.581146         time  2737.961182  -0.0451  -5.7503
904   Topic17    43.111671        point  1516.579224   0.4899  -5.8060

[1307 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
15262      6  0.981104       aaliya
245        1  0.169367       accord
245        2  0.097838       accord
245        3  0.119215       accord
245        4  0.078929       accord
...      ...       ...          ...
5532       4  0.988750          yen
13539     11  0.953447  yovanovitch
5987      14  0.948310         zack
20830      3  0.957729      zanskar
5988      14  0.928150    zimmerman

[4977 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 11, 5, 14, 3, 4, 7, 13, 16, 1, 8, 15, 17, 12, 10, 9, 6])

Finding the topic in each articles

In [149]:
corpus_lda_model = ldamodel[doc_term_matrix]
for article in corpus_lda_model[5]:
    print(article)

(4, 0.6090433)
(14, 0.20145865)
(16, 0.18348165)


For aritlce 5,the highest correlated topic was 4, which is: '0.024*"diamond" + 0.018*"ring" + 0.010*"cut" + 0.010*"set" + 0.008*"engag" + 0.006*"look" + 0.006*"gold" + 0.005*"band" + 0.005*"price" + 0.005*"ritani"

Let's do one final check based on our articles being ordered by length.

In [152]:
from operator import itemgetter
index_high_corr_list = []

index_counter = 0

for article in corpus_lda_model:
    highest_feature = max(article, key=itemgetter(1))[0]
    max_correlation = max(article, key=itemgetter(1))[1]
    index_high_corr_list.append([index_counter, highest_feature, max_correlation])
    index_counter += 1

index_high_corr_list[:5]

[[0, 9, 0.99734044], [1, 8, 0.5833368], [2, 2, 0.4316401], [3, 15, 0.99806225], [4, 0, 0.74044055]]

# Recommender Demonstration

 We initialize the similarity matrix between all BoW articles

In [202]:
index = similarities.MatrixSimilarity(ldamodel[doc_term_matrix])
Ind = pd.DataFrame(index)
Ind.shape

(3432, 3432)

In [203]:
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(Ind, Ind)

#drop duplicate titles, if any
indices = pd.Series(df.index, index=df['Article_id']).drop_duplicates()

In [204]:
# Function that takes in Article id as input and gives recommendations 
def content_recommender(Article_id, cosine_sim=cosine_sim, df=df, indices=indices):
    # Obtain the index of the article that matches the Article_id
    idx = indices[Article_id]

    # Get the pairwsie similarity scores of all articles with that article
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the articles based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar articles. Ignore the first article.
    sim_scores = sim_scores[1:11]

    # Get the article indices
    article_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['Content'].iloc[article_indices]

In [205]:
#Get recommendations for The article ids
content_recommender(2)

1287    There is both good news and some not so good n...
516     KEY POINTS $1 billion in loans went to publicl...
2929    Congress president Sonia Gandhi on Monday reje...
2450    A file picture of Sri Rama Sene president Pram...
1682    The Committee for Consultations on the situati...
849     Farmers in China are being offered cash to qui...
596     Gilead Sciences Inc said on Tuesday, May 12 it...
1078    KEY POINTS "Paralives" practically similar to ...
1485    BRIMMING WITH IDEAS: From left: S.B. Shukla, F...
2029    Bharatiya Janata Party State president V. Mura...
Name: Content, dtype: object

# Using TfIdfVectorizer 

In [235]:
#Import TfIdfVectorizer from the scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stopwords
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df['Content'] = df['Content'].fillna('')

#Construct the required TF-IDF matrix by applying the fit_transform method on the overview feature
tfidf_matrix = tfidf.fit_transform(df['Content'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(4432, 48366)

In [236]:
#drop duplicate titles, if any
indices = pd.Series(df.index, index=df['Article_id']).drop_duplicates(keep='first')

In [237]:
# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [238]:
# Function that takes in Article id as input and gives recommendations 
def content_recommender_2(Article_id, cosine_sim=cosine_sim, df=df, indices=indices):
    # Obtain the index of the article that matches the Article_id
    idx = indices[Article_id]

    # Get the pairwsie similarity scores of all articles with that article
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the articles based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar articles. Ignore the first article.
    sim_scores = sim_scores[1:11]

    # Get the article indices
    article_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['Content'].iloc[article_indices]

In [234]:
#Get recommendations for The article ids
content_recommender_2(4)

614     KEY POINTS Trump will not shut down America ag...
46      US President Donald Trump initially wanted to ...
3459    US President Donald Trump initially wanted to ...
3681    US President Donald Trump initially wanted to ...
3917    US President Donald Trump initially wanted to ...
124     The World Health Organization on Wednesday rep...
824     The World Health Organization on Wednesday rep...
4168    The World Health Organization on Wednesday rep...
781     The World Health Organization has reported the...
62      The World Health Organization on Wednesday rep...
Name: Content, dtype: object